Импортируем нужные модули

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

в зависимости от расположения файла базы данных создаем подключение и производим запрос к базе

In [6]:
query = """
    SELECT u.user_id, u.lt_day, 
            (CASE 
                WHEN u.lt_day > 365 THEN 'FALSE'
                ELSE 'TRUE'
            END) AS is_new,     
    u.age, 
    gender_segment,    
    u.os_name, u.cpe_type_name, l.country, l.city, 
    SUBSTRING(ags.title, 4) AS age_segment, SUBSTRING(ts.title, 4) AS traffic_segment, 
    SUBSTRING(ls.title, 4) AS lifetime_segment, u.nps_score,
        CASE 
            WHEN u.nps_score > 8 THEN 'promoters'
            WHEN u.nps_score > 6 THEN 'passives'
            ELSE 'detractors'
        END AS nps_group     
    FROM user AS u
    LEFT JOIN location AS l ON u.location_id = l.location_id
    LEFT JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
    LEFT JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
    LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
    """

In [7]:
if os.path.exists('telecomm_csi.db'):
    engine = create_engine('sqlite:///telecomm_csi.db', echo = False)
    df = pd.read_sql(query, engine)
elif os.path.exists(r'/datasets/telecomm_csi.db'):
    engine = create_engine('sqlite:////datasets/telecomm_csi.db', echo = False)
    df = pd.read_sql(query, engine)
else:
   raise "Something wrong!"

Проверяем выгрузку

In [4]:
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,FALSE,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,FALSE,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,FALSE,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,FALSE,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,FALSE,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


Сохраняем выгрузку в файл csv для дальнейшей работы:

In [5]:
df.to_csv('survey.csv', index=False)

Дашборд: 
https://public.tableau.com/views/2_16733788281260/sheet25_2?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

и презентация:
https://disk.yandex.ru/i/v9wnmEp9SUzX6g